# Qu'est-ce qu'une machine à états finis, MEF (Finite State Machine, FSM) ?

C'est une façon de décrire un système ou un processus qui peut être dans un état (**state**) à la fois et change d'état quand un évènement (**input**) se produit. À chaque changement (**transition**), la machine peut produire une action (**output**).

Exemple de FSM: fantôme du jeu **Pacman**

![fsm](./images/fsm_pacman.png)

# Les ingrédients d’une FSM

<details>
  <summary><b>États (States)</b></summary>
  Les « modes » possibles du système (ex. CHASSE, MORT, FUITE).
</details>

<br/>

<details>
  <summary><b>Entrées (Inputs)</b></summary>
  Ce qui arrive au système (ex. pièce insérée, bouton pressé).
</details>

<br/>

<details>
  <summary><b>Transitions</b></summary>
  Règles qui gouvernent le passage entre états (ex. si on reçoit l'entrée X en état Y, aller à l’état Z).
</details>

<br/>

<details>
  <summary><b>Sorties (Outputs)</b></summary>
  Ce que le système fait (ex. chasser Pacman, réapparaitre au centre de la carte).
</details>

<br/>

<details>
  <summary><b>Pour aller plus loin</b></summary>
  Il existe 2 familles de MEF: <b>Moore</b> (sortie dépend de l’état) et <b>Mealy</b> (sortie dépend de l’état et de l’entrée).
</details>

# Code ta première FSM

In [ ]:
from typing import Any, Dict


def rouler_fsm(fsm: Dict[str, Dict[str, Any]], etat_initial: str) -> None:
    """
    Cette fonction permet de faire rouler une machine à états finis,
    à partir d'un état initial donné et d'entrées utilisateur.

    Arguments:
        fsm :   La machine à états finis, représentée comme un dictionnaire.
                Chaque clé est un état et chaque valeur est un sous-dictionnaire
                liant les entrées possibles aux états suivants.
        etat_initial : L'état initial de la machine à états finis.
    """

    if etat_initial not in fsm:
        print("ERREUR: L'état initial n'existe pas dans la machine à états finis.")
        return

    etat_actuel = etat_initial
    print(f"État actuel: {etat_actuel}")
    while True:
        entree = input("Nouvelle entrée: ")
        if entree == "":
            break
        else:
            print("=" * 40)
            print(f"Entrée reçue: {entree}")
            entree = entree.lower()
            if entree in fsm[etat_actuel]:
                # Point le plus important: la transition d'état
                # On accède aux entrées possibles pour l'état actuel (fsm[etat_actuel])
                # puis on utilise l'entrée de l'utilisateur pour passer à l'état suivant (fsm[etat_actuel][entree])
                etat_actuel = fsm[etat_actuel][entree]
                print(f"État actuel: {etat_actuel}")
            else:
                print("Transition invalide pour cet état/entrée. Veuillez réessayer.")


# Définition de la machine à états finis pour le comportement d'un fantôme dans Pacman
# On code la fsm comme dictionnaire:
    # Chaque état possible est une clé
    # Chaque valeur est un sous-dictionnaire qui lie les entrées possibles aux états suivants
    # Par exemple, dans l'état "fuite", si l'entrée est "fin powerup", on passe à l'état "chasse"
    # mais si l'entrée est "pacman attrape", on passe à l'état "mort"
fsm = {
    "chasse": {
        "pacman powerup": "fuite"
    },
    "fuite": {
        "fin powerup": "chasse",
        "pacman attrape": "mort"
    },
    "mort": {
        "respawn atteint": "chasse"
    }
}

rouler_fsm(fsm, etat_initial="chasse")

# Et si on voulait arrêter et recommencer le jeu ?

On ajoute un nouvel état « **en attente** » à notre machine! Les transitions vers cet état correspondent à l'entrée « **fin jeu** ».

In [ ]:
# Copie du fsm existant
nouvelle_fsm = fsm.copy()

# On rajoute des transitions pour y aller à partir de
# tous les autres états (on peut arrêter à tout moment),
# quand l'utilisateur entre "fin jeu"
for etat in fsm.keys():
    nouvelle_fsm[etat]["fin jeu"] = "en attente"

# On rajoute l'état "en attente" et la transition pour en sortir
# L'entrée "jeu commence" nous fait retourner à l'état "chasse"
nouvelle_fsm["en attente"] = {
    "jeu commence": "chasse"
}

rouler_fsm(nouvelle_fsm, etat_initial="en attente")

# Pourquoi c'est utile une FSM ?

- On décrit un **système complexe** avec quelques **blocs simples** (états, entrées, transitions, sorties), même s’il y a des dizaines d’états et des centaines de transitions.

- Chaque état a un **rôle clair** et chaque transition explique **quand** et **pourquoi on change**.

- Pour chaque situation, la règle « **si… alors…** » est écrite: **moins d'erreurs**, **moins de cas oubliés**.

- Facile à **tester** et à **faire évoluer** : on peut ajouter un **nouvel état** ou une **nouvelle règle** sans casser le reste.

# Mauvaises et bonnes pratiques

### Qu'arrive t-il si une chaîne de caractères qui représente un état dans le dictionnaire est erronée ?

In [ ]:
fsm_erronee = {
    "en attente": {
        "jeu commence": "chase"  # "chase" au lieu de "chasse"
    },
    "chasse": {
        "pacman powerup": "fuite",
        "fin jeu": "en attente"
    },
    "fuite": {
        "fin powerup": "chasse",
        "pacman attrape": "mort",
        "fin jeu": "en attente"
    },
    "mort": {
        "respawn atteint": "chasse",
        "fin jeu": "en attente"
    }
}

rouler_fsm(fsm_erronee, etat_initial="en attente")

### C'est bon de définir chaque état avec sa propre variable!

In [ ]:
en_attente = "en attente"
chasse = "chasse"
fuite = "fuite"
mort = "mort"

# et pourquoi pas les entrées aussi?
jeu_commence = "jeu commence"
pacman_powerup = "pacman powerup"
fin_powerup = "fin powerup"
pacman_attrape = "pacman attrape"
respawn_atteint = "respawn atteint"
fin_jeu = "fin jeu"


fsm_bonne_pratique = {
    en_attente: {
        jeu_commence: chasse
    },
    chasse: {
        pacman_powerup: fuite,
        fin_jeu: en_attente
    },
    fuite: {
        pacman_powerup: chasse,
        fin_powerup: mort,
        fin_jeu: en_attente
    },
    mort: {
        respawn_atteint: chasse,
        fin_jeu: en_attente
    }
}

rouler_fsm(fsm_bonne_pratique, etat_initial=en_attente)

### C'est encore mieux de les définir comme énumération (**Enum**)

Les énumérations nous permettent d'associer à des noms des valeurs uniques. C'est plus facile à maintenir, à lire et ça réduit le risque d'erreurs.

In [ ]:
from enum import Enum

# Ici nos états et entrées sont définis comme des énumérations.
# Les valeurs sont des entiers uniques, qui nous permettent des évaluations/comparaisons rapides et efficaces.
# (ex. comparaison entre deux entiers 1 et 3,
# au lieu de comparer deux chaînes de caractères "pacman powerup" et "pacman attrape")
class Etats(Enum):
    EN_ATTENTE = 0
    CHASSE = 1
    FUITE = 2
    MORT = 3

class Entrees(Enum):
    JEU_COMMENCE = 0
    PACMAN_POWERUP = 1
    FIN_POWERUP = 2
    PACMAN_ATTRAPE = 3
    RESPAWN_ATTEINT = 4
    FIN_JEU = 5


fsm_avec_enum = {
    Etats.EN_ATTENTE: {
        Entrees.JEU_COMMENCE: Etats.CHASSE
    },
    Etats.CHASSE: {
        Entrees.PACMAN_POWERUP: Etats.FUITE,
        Entrees.FIN_JEU: Etats.EN_ATTENTE
    },
    Etats.FUITE: {
        Entrees.FIN_POWERUP: Etats.CHASSE,
        Entrees.PACMAN_ATTRAPE: Etats.MORT,
        Entrees.FIN_JEU: Etats.EN_ATTENTE
    },
    Etats.MORT: {
        Entrees.RESPAWN_ATTEINT: Etats.CHASSE,
        Entrees.FIN_JEU: Etats.EN_ATTENTE
    }
}

### Par contre, il faut redéfinir la fonction **rouler_fsm** pour que cette fois-ci elle supporte des enumérations!

In [ ]:
from typing import Dict

def rouler_fsm_enum(fsm: Dict[Etats, Dict[Entrees, Etats]], etat_initial: Etats) -> None:
    """
    Cette fonction permet de faire rouler une machine à états finis,
    à partir d'un état initial donné et d'entrées utilisateur.

    Arguments:
        fsm :   La machine à états finis, représentée comme un dictionnaire.
                Chaque clé est un état (Etats) et chaque valeur est un sous-dictionnaire
                liant les entrées possibles (Entrees) aux états suivants (Etats).
        etat_initial : L'état initial de la machine à états finis (Etats).
    """

    if etat_initial not in fsm:
        print("ERREUR: L'état initial n'existe pas dans la machine à états finis.")
        return

    etat_actuel = etat_initial
    print(f"État actuel: {etat_actuel.name.lower().replace('_', ' ')}")
    while True:
        entree_str = input("Nouvelle entrée: ")
        if entree_str == "":
            break
        else:
            print("=" * 30)
            print(f"Entrée reçue: {entree_str}")
            try:
                entree = Entrees[entree_str.upper().replace(" ", "_")]
            except KeyError:
                print("Entrée invalide. Veuillez réessayer.")
                continue

            if entree in fsm[etat_actuel]:
                etat_actuel = fsm[etat_actuel][entree]
                print(f"État actuel: {etat_actuel.name.lower().replace('_', ' ')}")
            else:
                print("Transition invalide pour cet état/entrée. Veuillez réessayer.")

rouler_fsm_enum(fsm_avec_enum, etat_initial=Etats.EN_ATTENTE)

### Et si on avait des dizaines d'états et une centaine de transitions ?

Le dictionnaire est toujours une bonne idée ? Est-ce que c'est facile à lire et à maintenir dans ce cas ?

In [ ]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), "..", ".."))
from ai.fsm.core import Machine

# Toute la logique derrière une FSM peut être cachée (encapsulée) derrière une classe.

fantome_fsm = Machine(states=Etats, initial_state=Etats.EN_ATTENTE)

# On ajoute les transitions une par une
fantome_fsm.add_transition(sources=Etats.CHASSE, dest=Etats.FUITE, trigger=Entrees.PACMAN_POWERUP)
fantome_fsm.add_transition(sources=Etats.FUITE, dest=Etats.CHASSE, trigger=Entrees.FIN_POWERUP)
fantome_fsm.add_transition(sources=Etats.FUITE, dest=Etats.MORT, trigger=Entrees.PACMAN_ATTRAPE)
fantome_fsm.add_transition(sources=Etats.MORT, dest=Etats.CHASSE, trigger=Entrees.RESPAWN_ATTEINT)

# On ajoute les transitions pour arrêter et recommencer le jeu
fantome_fsm.add_transition(
    sources=[Etats.CHASSE, Etats.FUITE, Etats.MORT],
    dest=Etats.EN_ATTENTE,
    trigger=Entrees.FIN_JEU
)
fantome_fsm.add_transition(sources=Etats.EN_ATTENTE, dest=Etats.CHASSE, trigger=Entrees.JEU_COMMENCE)

La classe **Machine** nous permet d'appeler des **méthodes** qui passe l'entrée au FSM! Ces méthodes ont le même nom que les **entrées** (triggers) que nous avons définies.

In [ ]:
def format_etat(nom: str) -> str:
    return nom.lower().replace('_', ' ')

fantome_fsm.reset()  # On remet la FSM à son état initial si c'est pas déjà fait
print(f"État initial: {format_etat(fantome_fsm.current_state.name)}")

fantome_fsm.jeu_commence()  # On appelle la méthode qui correspond à l'entrée "jeu commence"
print(f"\nÉtat après 'jeu commence': {fantome_fsm.current_state.name}")

fantome_fsm.pacman_powerup()
print(f"\nÉtat après 'pacman powerup': {fantome_fsm.current_state.name}")

fantome_fsm.pacman_attrape()
print(f"\nÉtat après 'pacman attrape': {fantome_fsm.current_state.name}")

fantome_fsm.respawn_atteint()
print(f"\nÉtat après 'respawn atteint': {fantome_fsm.current_state.name}")

fantome_fsm.fin_jeu()
print(f"\nÉtat après 'fin jeu': {fantome_fsm.current_state.name}")

La classe **Machine** nous permet aussi de visualiser notre FSM! L'**état actuel** est représenté par un **double cercle**.

In [ ]:
display(fantome_fsm.to_graphviz())

Qu'est-ce qui se passe si on essaie de faire une transition non valide? Par exemple, passer de l'état **chasse** vers **mort** avec l'entrée **pacman_attrape**.

In [ ]:
fantome_fsm.reset()
print(f"État initial: {format_etat(fantome_fsm.current_state.name)}")

success = fantome_fsm.jeu_commence()
print(f"\nÉtat après 'jeu commence': {fantome_fsm.current_state.name}")
print(f"Transition réussie: {success}")

success = fantome_fsm.pacman_attrape()  # Transition invalide
print(f"\nÉtat après 'pacman attrape': {fantome_fsm.current_state.name}")
print(f"Transition réussie: {success}")

# Pourquoi pas se créer un petit jeu avec notre nouvelle classe **Machine** ?

<h1>🦆 cross the 🛣️</h1>

![crossy_road](./images/crossy_road.gif)

## Planifions d'abord notre machine à états finis!

![duck_fsm](./images/fsm_duck.svg)

## Est-ce qu'on est capable de coder les **états** et les **entrées** ?

In [ ]:
from enum import Enum

# Combien d'états nous avons dans notre FSM au total ? Complétez l'énumération ci-dessous.
class EtatsCanard(Enum):
    EN_ATTENTE = 0
    AVANCE = 1

# Même chose pour les entrées!
class EntreesCanard(Enum):
    AUCUN_BOUTON = 0
    FLECHE_HAUT = 1

## Est-ce qu'on est capable de **créer une FSM** avec la classe **Machine** ?

In [ ]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), "..", ".."))
from ai.fsm.core import Machine

canard_fsm = ... # À remplacer

## Ajouter les transitions ?

In [ ]:
# Combien de transitions devons-nous ajouter ? Complétez ci-dessous.
canard_fsm.add_transition(
    sources=EtatsCanard.EN_ATTENTE,
    dest=EtatsCanard.AVANCE,
    trigger=EntreesCanard.FLECHE_HAUT
)

...

## Vérifions si la machine correspond à notre plan!

In [ ]:
display(canard_fsm.to_graphviz())

## Solution complète

In [ ]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), "..", ".."))

from enum import Enum

from ai.fsm.core import Machine


# On définit les états et entrées pour notre jeu de traversée de route
class EtatsCanard(Enum):
    EN_ATTENTE = 0
    AVANCE = 1
    TOURNE_GAUCHE = 2
    TOURNE_DROITE = 3
    MORT = 4

class EntreesCanard(Enum):
    AUCUN_BOUTON = 0
    FLECHE_HAUT = 1
    FLECHE_GAUCHE = 2
    FLECHE_DROITE = 3
    COLLISION = 4

# On crée la machine à états finis pour le canard
duck_fsm = Machine(states=EtatsCanard, initial_state=EtatsCanard.EN_ATTENTE)

# À n'importe quel autre état, la canard peut passer à l'état MORT en cas de COLLISION
duck_fsm.add_transition(
    sources=[
        EtatsCanard.EN_ATTENTE,
        EtatsCanard.AVANCE,
        EtatsCanard.TOURNE_GAUCHE,
        EtatsCanard.TOURNE_DROITE
    ],
    dest=EtatsCanard.MORT,
    trigger=EntreesCanard.COLLISION
)

# À partir de l'état EN_ATTENTE, le canard peut avancer ou tourner selon la flèche appuyée
duck_fsm.add_transition(
    sources=EtatsCanard.EN_ATTENTE,
    dest=EtatsCanard.AVANCE,
    trigger=EntreesCanard.FLECHE_HAUT
)
duck_fsm.add_transition(
    sources=EtatsCanard.EN_ATTENTE,
    dest=EtatsCanard.TOURNE_GAUCHE,
    trigger=EntreesCanard.FLECHE_GAUCHE
)
duck_fsm.add_transition(
    sources=EtatsCanard.EN_ATTENTE,
    dest=EtatsCanard.TOURNE_DROITE,
    trigger=EntreesCanard.FLECHE_DROITE
)

# Si aucun bouton n'est appuyé, le canard retourne à l'état EN_ATTENTE s'il était en mouvement
duck_fsm.add_transition(
    sources=[
        EtatsCanard.AVANCE,
        EtatsCanard.TOURNE_GAUCHE,
        EtatsCanard.TOURNE_DROITE
    ],
    dest=EtatsCanard.EN_ATTENTE,
    trigger=EntreesCanard.AUCUN_BOUTON
)

display(duck_fsm.to_graphviz())

## Et maintenant on s'amuse!

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [ ]:
from examples.workshop_0.src.duck_game import DuckGame

duck_fsm.reset()
game = DuckGame(
    duck_fsm,
    trigger_map={
        "ArrowUp": "fleche_haut",
        "ArrowLeft": "fleche_gauche",
        "ArrowRight": "fleche_droite",
    },
    release_trigger="aucun_bouton",
    goal_trigger="arrive",
    hit_trigger="collision",
    starting_lane_count=4,
    starting_max_obstacles_per_lane=3
)
if not IN_COLAB:
    game.start()

In [ ]:
display(game.canvas)

if IN_COLAB:
    import ipywidgets as W

    play = W.Play(interval=1, value=0, min=0, max=10**9, step=1)  # long “infinite” counter
    slider = W.IntSlider(min=0, max=10**9, step=1)  # mirror for .observe
    W.jslink((play, "value"), (slider, "value"))

    def on_tick(change):
        game._draw()

    slider.observe(on_tick, names="value")

    display(W.HBox([play]))
    on_tick({"new": 0})